# Getting started with the Azure Agent Service with the OpenAI SDK

## Requirements


In [ ]:
%pip install azure-ai-projects
%pip install azure-identity
%pip install python-dotenv
%pip install openai

In [ ]:
import os, time

from pprint import pp as pp

from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import CodeInterpreterTool, BingGroundingTool

from openai import AzureOpenAI

# Load environment variables from .env file
load_dotenv()

# Print the environments we will be using.
print(f"PROJECT_CONNECTION_STRING: {os.getenv('PROJECT_CONNECTION_STRING')}")
print(f"BING_CONNECTION_NAME: {os.getenv('BING_CONNECTION_NAME')}")  

Let's define some auxiliary functions we will use during this lab. One to do pretty print of the several objects agent service uses, and another way to execute an agent.

In [ ]:
def pprint(obj):    
    pp(obj.as_dict() if hasattr(obj, "as_dict") else obj, width=100)

def run(thread, assistant):
    # run = project.agents.create_run(thread_id=thread.id, assistant_id=assistant.id)
    run = project.agents.create_and_process_run(thread_id=thread.id, assistant_id=assistant.id)

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project.agents.get_run(thread_id=thread.id, run_id=run.id)

        if run.status not in ["queued", "in_progress", "requires_action"]:
            break

    return run  

### Connecting to a project

Agents are created in the context of an [Azure AI Foundry project](https://learn.microsoft.com/en-us/azure/ai-studio/concepts/architecture). In order to create one, first we need to connect to the project using a connection string provided in the Azure Foundry Portal.

 You can also find your connection string in the overview for your project in the Azure AI Foundry portal, under Project details > Project connection string.
![Connection String](./assets/img/portal-connection-string.png "Connection String")

Get a project reference using the current authenticated user and the connection string to the project where the agents will be created. 

In [ ]:
project = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

openai_client: AzureOpenAI = project.inference.get_azure_openai_client(
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
)

In [ ]:
for a in project.agents.list_agents().data:    
    project.agents.delete_agent(a.id)
    
for a in openai_client.beta.assistants.list():
    openai_client.beta.assistants.delete(a.id)

### Creating Agents

Now that we are connected to the project, let's create a very simple agent using **GPT-4o-mini** model as the agent _"brain"_. We are also providing instructions on how the agent should behave, adding a description and custom metadata fields. These metadata fields can be used to add custom information to find agents later for scenario with several agents.

In [ ]:
portfolio_agent = project.agents.create_agent(
    model="gpt-4o-mini",
    name="Portfolio Guidance",
    instructions="You are a financial agent, providing portfolio guidance for users.",
    description="This agent was created to provide guidance for our users.",
    metadata= {
        "department": "finance",
        "owner": "jdoe"
    }
)
pprint(portfolio_agent)

Every agent is unique and has an id, which starts with the string _asst_. This id and can be used to retrieve the agent later.

Let's create a second agent for the marketing team, but this one will be created using the OpenAI SDK. Even supporting the OpenAI SDK, the recommended SDK is the Agent Service SDK.
Please, note that 

In [ ]:
branding_agent = openai_client.beta.assistants.create(
    model="gpt-4o-mini",
    name="Branding Guidance",
    instructions="You are a marketing agent, providing guidance about the correct way to use our branding. Make sure to warn users that our logo must be at the same size as the other company logos."
)
pprint(branding_agent)

Using the project object it is possible to list all agents in the project, delete an agent or update an agent.

Right now we should have two agents created in the project.

In [ ]:
pprint(project.agents.list_agents())

In [ ]:
pprint(openai_client.beta.assistants.list())

Let's improve the instructions we provided for the portfolio agent.

In [ ]:
portfolio_agent = project.agents.update_agent(portfolio_agent.id, instructions="You are a financial agent, providing portfolio guidance for users. Avoid using too technical terms, and be clear about the risks.")
pprint(portfolio_agent)

### Chatting with the Agent

In order to chat with an agent, we need to create a **thread**. A thread is a **container** for messages, and there are two types of messages: messages created by the user or created by an agent.

**Important!**
1. If you don't save the thread id, you won't be able to recover the thread later. Keep this in mind, and save it to another database.
2. A thread is not tied to an agent. **The same thread can be used by several agents**.
3. Unless deleted manually, threads are kept forever.

Jane wants to get information on how to invest her money, so let's create a thread for Jane.
You can also add metadata for threads, and in this case we are adding the user id from Microsoft Entra.

In [ ]:
thread_jane = project.agents.create_thread(
    metadata= {
        "entraUserId": "a1ddfd30-8420-4a8d-b155-4b5f05998d69"
    }
)
pprint(thread_jane)

In [ ]:
message = project.agents.create_message(
    thread_id=thread_jane.id,
    role="user",
    content="What is the best option to invest my money?",
)
pprint(message)

In [ ]:
pprint(project.agents.list_messages(thread_jane.id))

At this point, we created a thread containing only one message. As you can see, the thread or message are not linked to any assistant. You can validate that looking the **run_id** and **assistant_id** properties for messages where the role is **user**.

Now, it's time to ask the agent to reason over the message the user sent and generate a reply. This is represented by a run object. To create a run we nee

Let's create a simple Python function called run to abstract all

In [ ]:
pprint(run(thread_jane, portfolio_agent))

In [ ]:
pprint(project.agents.list_messages(thread_jane.id))

Now, let's add a new message to the same thread.

In [ ]:
message = project.agents.create_message(
    thread_id=thread_jane.id,
    role="user",
    content="What are the branding guidelines I need to follow?",
)
pprint(message)

Let's see the 

In [ ]:
pprint(project.agents.list_messages(thread_jane.id))

In [ ]:
pprint(run(thread_jane, branding_agent))

In [ ]:
pprint(project.agents.list_messages(thread_jane.id))

## Giving arms and legs to agents with tools

An agent without any tool

### Generating and executing code with Code Interpreter



In [ ]:
code_interpreter = CodeInterpreterTool()

portfolio_agent = project.agents.update_agent(portfolio_agent.id, tools=code_interpreter.definitions, tool_resources=code_interpreter.resources)
pprint(portfolio_agent)

In [ ]:
thread_bill = project.agents.create_thread(
    metadata= {
        "entraUserId": "444dfd30-8420-4a8d-b155-4b5f05994545"
    }
)
pprint(thread_bill)

In [ ]:
message = project.agents.create_message(
    thread_id=thread_bill.id,
    role="user",
    content="Create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million",
)
pprint(message)

In [ ]:
pprint(project.agents.list_messages(thread_bill.id))

In [ ]:
pprint(run(thread_bill, portfolio_agent))

In [ ]:
pprint(project.agents.list_messages(thread_bill.id))

In [ ]:
pprint(project.agents.list_files())

### Retrieving data from Internet using Bing Search

In [ ]:
bing_connection = project.connections.get(
    connection_name="api"
)

bing = BingGroundingTool(connection_id=bing_connection.id)

In [ ]:
news_agent = project.agents.create_agent(
    model="gpt-4o-2024-08-06",
    name="News",
    instructions="You are a news agent, providing updated information for our users.",
    description="This agent was created to provide updated news for our users.",
    tools=bing.definitions,
    metadata= {
        "department": "marketing",
        "owner": "joanna"
    }
)
pprint(news_agent)

In [ ]:
thread_alice = project.agents.create_thread(
    metadata= {
        "entraUserId": "444dfd30-8420-4a8d-b155-4b5f05994545"
    }
)
pprint(thread_alice)

In [ ]:
message = project.agents.create_message(
    thread_id=thread_alice.id,
    role="user",
    content="What is the latest news about quantum computing?",
)
pprint(message)

In [ ]:
pprint(run(thread_alice, news_agent))

In [ ]:
pprint(project.agents.list_messages(thread_alice.id))

### Using a custom function